In [ ]:
# EfficientNet
# ==============================
# Federated Ocular Disease Classifier with Flower
# With improvements:
# - Weighted loss
# - Lower threshold
# - More epochs per round
# - Separate per-client and global CSVs
# ==============================

import os
import pandas as pd
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision import transforms as T
from PIL import Image
from efficientnet_pytorch import EfficientNet
import flwr as fl
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import ast

# Paths
base_path = "C:/Users/aitoo/OneDrive/Desktop/FYP-Code/ODIR-5K/ODIR-5K"
train_images_dir = os.path.join(base_path, "Training Images")
data_file = os.path.join(base_path, "data.xlsx")

# Load data
df = pd.read_excel(data_file)
label_cols = ["N","D","G","C","A","H","M","O"]
rows = []
for _, r in df.iterrows():
    rows.append({"filename": r["Left-Fundus"], "labels": r[label_cols].values.astype(int).tolist()})
    rows.append({"filename": r["Right-Fundus"], "labels": r[label_cols].values.astype(int).tolist()})
df_images = pd.DataFrame(rows)
df_images.to_csv("C:/Users/aitoo/OneDrive/Desktop/FYP-Code/first_raw/ocular_labels.csv", index=False)

# Reload and parse labels
df_images = pd.read_csv("C:/Users/aitoo/OneDrive/Desktop/FYP-Code/first_raw/ocular_labels.csv")
df_images["labels"] = df_images["labels"].apply(ast.literal_eval)

# Dataset
class OcularDataset(Dataset):
    def __init__(self, dataframe, image_dir, transform=None):
        self.df = dataframe.reset_index(drop=True)
        self.image_dir = image_dir
        self.transform = transform if transform else T.Compose([
            T.Resize((224, 224)),
            T.ToTensor(),
            T.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ])

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        img_path = os.path.join(self.image_dir, row['filename'])
        img = Image.open(img_path).convert("RGB")
        labels = torch.tensor(row["labels"], dtype=torch.float32)
        return self.transform(img), labels

# Split dataset into 5 clients
total_len = len(df_images)
split_size = total_len // 5
splits = [split_size] * 5
splits[-1] += total_len - sum(splits)
clients_dfs = random_split(df_images, splits)

# Model
def get_model():
    model = EfficientNet.from_pretrained("efficientnet-b0")
    model._fc = nn.Linear(model._fc.in_features, 8)
    return model

# Device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Client class
class FlowerClient(fl.client.NumPyClient):
    def __init__(self, model, train_loader, val_loader, client_id):
        self.model = model.to(device)
        self.train_loader = train_loader
        self.val_loader = val_loader
        pos_weight = torch.tensor([3.0]*8).to(device)  # Weighted loss for imbalance
        self.criterion = nn.BCEWithLogitsLoss(pos_weight=pos_weight)
        self.optimizer = torch.optim.Adam(self.model.parameters(), lr=1e-4)
        self.client_id = client_id
        self.round_counter = 0  # To track rounds

    def get_parameters(self, config):
        return [val.cpu().numpy() for _, val in self.model.state_dict().items()]

    def set_parameters(self, parameters):
        state_dict = self.model.state_dict()
        new_state_dict = {k: torch.tensor(v).to(device) for k, v in zip(state_dict.keys(), parameters)}
        self.model.load_state_dict(new_state_dict)

    def fit(self, parameters, config):
        self.set_parameters(parameters)
        self.model.train()
        for epoch in range(3):  # Train longer per round
            for images, labels in self.train_loader:
                images, labels = images.to(device), labels.to(device)
                self.optimizer.zero_grad()
                outputs = self.model(images)
                loss = self.criterion(outputs, labels)
                loss.backward()
                self.optimizer.step()
        return self.get_parameters(config={}), len(self.train_loader.dataset), {}

    def evaluate(self, parameters, config):
        self.set_parameters(parameters)
        self.model.eval()
        y_true, y_pred = [], []
        val_loss = 0
        with torch.no_grad():
            for images, labels in self.val_loader:
                images, labels = images.to(device), labels.to(device)
                outputs = self.model(images)
                val_loss += self.criterion(outputs, labels).item()
                preds = torch.sigmoid(outputs).cpu().numpy()
                y_true.extend(labels.cpu().numpy())
                y_pred.extend((preds > 0.3).astype(int))  # Lower threshold

        acc = accuracy_score(y_true, y_pred)
        prec = precision_score(y_true, y_pred, average="samples", zero_division=0)
        rec = recall_score(y_true, y_pred, average="samples", zero_division=0)
        f1 = f1_score(y_true, y_pred, average="samples", zero_division=0)

        self.round_counter += 1

        metrics_row = {
            "client": self.client_id,
            "round": self.round_counter,
            "loss": val_loss/len(self.val_loader),
            "accuracy": acc,
            "precision": prec,
            "recall": rec,
            "f1": f1
        }

        print(f"Client {self.client_id} - Round {self.round_counter} - Loss: {metrics_row['loss']:.4f} | Acc: {acc:.4f} | Prec: {prec:.4f} | Rec: {rec:.4f} | F1: {f1:.4f}")

        client_csv_path = f"C:/Users/aitoo/OneDrive/Desktop/FYP-Code/first_raw/FL_Client_{self.client_id}.csv"
        pd.DataFrame([metrics_row]).to_csv(
            client_csv_path,
            mode="a",
            header=not os.path.exists(client_csv_path),
            index=False,
        )

        return float(metrics_row["loss"]), len(self.val_loader.dataset), {}

# Client factory
def client_fn(cid: str):
    idx = int(cid)
    client_df = clients_dfs[idx].dataset.iloc[clients_dfs[idx].indices]
    dataset = OcularDataset(client_df, train_images_dir)
    split = int(0.8 * len(dataset))
    train_ds, val_ds = torch.utils.data.random_split(dataset, [split, len(dataset) - split])
    train_loader = DataLoader(train_ds, batch_size=8, shuffle=True)
    val_loader = DataLoader(val_ds, batch_size=8)
    return FlowerClient(get_model(), train_loader, val_loader, client_id=cid)

# Custom strategy with global evaluation
class MyFedAvg(fl.server.strategy.FedAvg):
    def evaluate(self, server_round, parameters):
        model = get_model().to(device)
        state_dict = model.state_dict()
        new_state_dict = {k: torch.tensor(v).to(device) for k, v in zip(state_dict.keys(), parameters)}
        model.load_state_dict(new_state_dict)
        model.eval()
        all_y_true, all_y_pred = [], []
        total_loss = 0
        with torch.no_grad():
            for idx in range(5):
                client_df = clients_dfs[idx].dataset.iloc[clients_dfs[idx].indices]
                dataset = OcularDataset(client_df, train_images_dir)
                _, val_ds = torch.utils.data.random_split(dataset, [int(0.8 * len(dataset)), len(dataset)-int(0.8 * len(dataset))])
                val_loader = DataLoader(val_ds, batch_size=8)
                for images, labels in val_loader:
                    images, labels = images.to(device), labels.to(device)
                    outputs = model(images)
                    total_loss += nn.BCEWithLogitsLoss()(outputs, labels).item()
                    preds = torch.sigmoid(outputs).cpu().numpy()
                    all_y_true.extend(labels.cpu().numpy())
                    all_y_pred.extend((preds > 0.3).astype(int))

        acc = accuracy_score(all_y_true, all_y_pred)
        prec = precision_score(all_y_true, all_y_pred, average="samples", zero_division=0)
        rec = recall_score(all_y_true, all_y_pred, average="samples", zero_division=0)
        f1 = f1_score(all_y_true, all_y_pred, average="samples", zero_division=0)

        metrics_row = {
            "round": server_round,
            "loss": total_loss / len(all_y_true),
            "accuracy": acc,
            "precision": prec,
            "recall": rec,
            "f1": f1
        }

        print(f"GLOBAL - Round {server_round} - Loss: {metrics_row['loss']:.4f} | Acc: {acc:.4f} | Prec: {prec:.4f} | Rec: {rec:.4f} | F1: {f1:.4f}")

        global_csv_path = "C:/Users/aitoo/OneDrive/Desktop/FYP-Code/first_raw/FL_Global.csv"
        pd.DataFrame([metrics_row]).to_csv(
            global_csv_path,
            mode="a",
            header=not os.path.exists(global_csv_path),
            index=False,
        )

        return float(metrics_row["loss"]), {}

# Run simulation
strategy = fl.server.strategy.FedAvg(
    fraction_fit=1.0,
    min_fit_clients=5,
    min_available_clients=5,
)

fl.simulation.start_simulation(
    client_fn=client_fn,
    num_clients=5,
    client_resources={
        "num_cpus": 1,
        "num_gpus": 1.0
    },
    strategy=strategy,
    config=fl.server.ServerConfig(num_rounds=50)
)


In [ ]:
###################

In [ ]:
# mobilenet_v3_large
# ==============================
# Federated Ocular Disease Classifier with Flower
# With improvements:
# - Weighted loss
# - Lower threshold
# - More epochs per round
# - Separate per-client and global CSVs
# ==============================

import os
import pandas as pd
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision import transforms as T
from PIL import Image
from efficientnet_pytorch import EfficientNet
import flwr as fl
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import ast
from torchvision import models



# Paths
base_path = "C:/Users/aitoo/OneDrive/Desktop/FYP-Code/ODIR-5K/ODIR-5K"
train_images_dir = os.path.join(base_path, "Training Images")
data_file = os.path.join(base_path, "data.xlsx")

# Load data
df = pd.read_excel(data_file)
label_cols = ["N","D","G","C","A","H","M","O"]
rows = []
for _, r in df.iterrows():
    rows.append({"filename": r["Left-Fundus"], "labels": r[label_cols].values.astype(int).tolist()})
    rows.append({"filename": r["Right-Fundus"], "labels": r[label_cols].values.astype(int).tolist()})
df_images = pd.DataFrame(rows)
df_images.to_csv("C:/Users/aitoo/OneDrive/Desktop/FYP-Code/first_raw/ocular_labels.csv", index=False)

# Reload and parse labels
df_images = pd.read_csv("C:/Users/aitoo/OneDrive/Desktop/FYP-Code/first_raw/ocular_labels.csv")
df_images["labels"] = df_images["labels"].apply(ast.literal_eval)

# Dataset
class OcularDataset(Dataset):
    def __init__(self, dataframe, image_dir, transform=None):
        self.df = dataframe.reset_index(drop=True)
        self.image_dir = image_dir
        self.transform = transform if transform else T.Compose([
            T.Resize((224, 224)),
            T.ToTensor(),
            T.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ])

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        img_path = os.path.join(self.image_dir, row['filename'])
        img = Image.open(img_path).convert("RGB")
        labels = torch.tensor(row["labels"], dtype=torch.float32)
        return self.transform(img), labels

# Split dataset into 5 clients
total_len = len(df_images)
split_size = total_len // 5
splits = [split_size] * 5
splits[-1] += total_len - sum(splits)
clients_dfs = random_split(df_images, splits)

# Model
def get_model():
    model = models.mobilenet_v3_large(pretrained=True)
    model.classifier[3] = nn.Linear(model.classifier[3].in_features, 8)
    return model


# Device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Client class
class FlowerClient(fl.client.NumPyClient):
    def __init__(self, model, train_loader, val_loader, client_id):
        self.model = model.to(device)
        self.train_loader = train_loader
        self.val_loader = val_loader
        pos_weight = torch.tensor([3.0]*8).to(device)  # Weighted loss for imbalance
        self.criterion = nn.BCEWithLogitsLoss(pos_weight=pos_weight)
        self.optimizer = torch.optim.Adam(self.model.parameters(), lr=1e-4)
        self.client_id = client_id
        self.round_counter = 0  # To track rounds

    def get_parameters(self, config):
        return [val.cpu().numpy() for _, val in self.model.state_dict().items()]

    def set_parameters(self, parameters):
        state_dict = self.model.state_dict()
        new_state_dict = {k: torch.tensor(v).to(device) for k, v in zip(state_dict.keys(), parameters)}
        self.model.load_state_dict(new_state_dict)

    def fit(self, parameters, config):
        self.set_parameters(parameters)
        self.model.train()
        for epoch in range(3):  # Train longer per round
            for images, labels in self.train_loader:
                images, labels = images.to(device), labels.to(device)
                self.optimizer.zero_grad()
                outputs = self.model(images)
                loss = self.criterion(outputs, labels)
                loss.backward()
                self.optimizer.step()
        return self.get_parameters(config={}), len(self.train_loader.dataset), {}

    def evaluate(self, parameters, config):
        self.set_parameters(parameters)
        self.model.eval()
        y_true, y_pred = [], []
        val_loss = 0
        with torch.no_grad():
            for images, labels in self.val_loader:
                images, labels = images.to(device), labels.to(device)
                outputs = self.model(images)
                val_loss += self.criterion(outputs, labels).item()
                preds = torch.sigmoid(outputs).cpu().numpy()
                y_true.extend(labels.cpu().numpy())
                y_pred.extend((preds > 0.3).astype(int))  # Lower threshold

        acc = accuracy_score(y_true, y_pred)
        prec = precision_score(y_true, y_pred, average="samples", zero_division=0)
        rec = recall_score(y_true, y_pred, average="samples", zero_division=0)
        f1 = f1_score(y_true, y_pred, average="samples", zero_division=0)

        self.round_counter += 1

        metrics_row = {
            "client": self.client_id,
            "round": self.round_counter,
            "loss": val_loss/len(self.val_loader),
            "accuracy": acc,
            "precision": prec,
            "recall": rec,
            "f1": f1
        }

        print(f"Client {self.client_id} - Round {self.round_counter} - Loss: {metrics_row['loss']:.4f} | Acc: {acc:.4f} | Prec: {prec:.4f} | Rec: {rec:.4f} | F1: {f1:.4f}")

        client_csv_path = f"C:/Users/aitoo/OneDrive/Desktop/FYP-Code/first_raw/FL_Client_{self.client_id}.csv"
        pd.DataFrame([metrics_row]).to_csv(
            client_csv_path,
            mode="a",
            header=not os.path.exists(client_csv_path),
            index=False,
        )

        return float(metrics_row["loss"]), len(self.val_loader.dataset), {}

# Client factory
def client_fn(cid: str):
    idx = int(cid)
    client_df = clients_dfs[idx].dataset.iloc[clients_dfs[idx].indices]
    dataset = OcularDataset(client_df, train_images_dir)
    split = int(0.8 * len(dataset))
    train_ds, val_ds = torch.utils.data.random_split(dataset, [split, len(dataset) - split])
    train_loader = DataLoader(train_ds, batch_size=8, shuffle=True)
    val_loader = DataLoader(val_ds, batch_size=8)
    return FlowerClient(get_model(), train_loader, val_loader, client_id=cid)

# Custom strategy with global evaluation
class MyFedAvg(fl.server.strategy.FedAvg):
    def evaluate(self, server_round, parameters):
        model = get_model().to(device)
        state_dict = model.state_dict()
        new_state_dict = {k: torch.tensor(v).to(device) for k, v in zip(state_dict.keys(), parameters)}
        model.load_state_dict(new_state_dict)
        model.eval()
        all_y_true, all_y_pred = [], []
        total_loss = 0
        with torch.no_grad():
            for idx in range(5):
                client_df = clients_dfs[idx].dataset.iloc[clients_dfs[idx].indices]
                dataset = OcularDataset(client_df, train_images_dir)
                _, val_ds = torch.utils.data.random_split(dataset, [int(0.8 * len(dataset)), len(dataset)-int(0.8 * len(dataset))])
                val_loader = DataLoader(val_ds, batch_size=8)
                for images, labels in val_loader:
                    images, labels = images.to(device), labels.to(device)
                    outputs = model(images)
                    total_loss += nn.BCEWithLogitsLoss()(outputs, labels).item()
                    preds = torch.sigmoid(outputs).cpu().numpy()
                    all_y_true.extend(labels.cpu().numpy())
                    all_y_pred.extend((preds > 0.3).astype(int))

        acc = accuracy_score(all_y_true, all_y_pred)
        prec = precision_score(all_y_true, all_y_pred, average="samples", zero_division=0)
        rec = recall_score(all_y_true, all_y_pred, average="samples", zero_division=0)
        f1 = f1_score(all_y_true, all_y_pred, average="samples", zero_division=0)

        metrics_row = {
            "round": server_round,
            "loss": total_loss / len(all_y_true),
            "accuracy": acc,
            "precision": prec,
            "recall": rec,
            "f1": f1
        }

        print(f"GLOBAL - Round {server_round} - Loss: {metrics_row['loss']:.4f} | Acc: {acc:.4f} | Prec: {prec:.4f} | Rec: {rec:.4f} | F1: {f1:.4f}")

        global_csv_path = "C:/Users/aitoo/OneDrive/Desktop/FYP-Code/first_raw/FL_Global.csv"
        pd.DataFrame([metrics_row]).to_csv(
            global_csv_path,
            mode="a",
            header=not os.path.exists(global_csv_path),
            index=False,
        )

        return float(metrics_row["loss"]), {}

# Run simulation
strategy = fl.server.strategy.FedAvg(
    fraction_fit=1.0,
    min_fit_clients=5,
    min_available_clients=5,
)

fl.simulation.start_simulation(
    client_fn=client_fn,
    num_clients=5,
    client_resources={
        "num_cpus": 1,
        "num_gpus": 1.0
    },
    strategy=strategy,
    config=fl.server.ServerConfig(num_rounds=50)
)


	Instead, use the `flwr run` CLI command to start a local simulation in your Flower app, as shown for example below:

		$ flwr new  # Create a new Flower app from a template

		$ flwr run  # Run the Flower app in Simulation Mode

	Using `start_simulation()` is deprecated.

            This is a deprecated feature. It will be removed
            entirely in future versions of Flower.
        
INFO :      Starting Flower simulation, config: num_rounds=50, no round_timeout
2025-07-04 17:40:13,170	INFO worker.py:1771 -- Started a local Ray instance.
INFO :      Flower VCE: Ray initialized with resources: {'accelerator_type:G': 1.0, 'node:__internal_head__': 1.0, 'CPU': 24.0, 'object_store_memory': 14049384038.0, 'node:127.0.0.1': 1.0, 'memory': 28098768078.0, 'GPU': 1.0}
INFO :      Optimize your simulation with Flower VCE: https://flower.ai/docs/framework/how-to-run-simulations.html
INFO :      Flower VCE: Resources for each Virtual Client: {'num_cpus': 1, 'num_gpus': 1.0}
INFO :      Flo

(ClientAppActor pid=28220) Downloading: "https://download.pytorch.org/models/mobilenet_v3_large-8738ca79.pth" to C:\Users\aitoo/.cache\torch\hub\checkpoints\mobilenet_v3_large-8738ca79.pth


  0%|          | 0.00/21.1M [00:00<?, ?B/s]
  1%|          | 256k/21.1M [00:00<00:10, 2.17MB/s]
  5%|▌         | 1.12M/21.1M [00:00<00:03, 5.92MB/s]
  8%|▊         | 1.75M/21.1M [00:00<00:04, 4.62MB/s]
 11%|█         | 2.25M/21.1M [00:00<00:05, 3.44MB/s]
 13%|█▎        | 2.75M/21.1M [00:00<00:05, 3.34MB/s]
 15%|█▍        | 3.12M/21.1M [00:01<00:07, 2.43MB/s]
 17%|█▋        | 3.50M/21.1M [00:01<00:09, 2.01MB/s]
 18%|█▊        | 3.75M/21.1M [00:01<00:09, 1.84MB/s]
 19%|█▉        | 4.00M/21.1M [00:01<00:10, 1.73MB/s]
 20%|██        | 4.25M/21.1M [00:01<00:10, 1.61MB/s]
 21%|██▏       | 4.50M/21.1M [00:02<00:11, 1.54MB/s]
 22%|██▏       | 4.75M/21.1M [00:02<00:11, 1.49MB/s]
 24%|██▎       | 5.00M/21.1M [00:02<00:11, 1.46MB/s]
 25%|██▍       | 5.25M/21.1M [00:02<00:12, 1.36MB/s]
 26%|██▌       | 5.50M/21.1M [00:02<00:12, 1.31MB/s]
 27%|██▋       | 5.75M/21.1M [00:03<00:11, 1.36MB/s]
 28%|██▊       | 6.00M/21.1M [00:03<00:11, 1.34MB/s]
 30%|██▉       | 6.25M/21.1M [00:03<00:11, 1.36MB/s]
 31

(ClientAppActor pid=28220) Client 3 - Round 1 - Loss: 0.5183 | Acc: 0.0464 | Prec: 0.3840 | Rec: 0.9077 | F1: 0.5220


(ClientAppActor pid=28220) WARNING :   DEPRECATED FEATURE: `client_fn` now expects a signature `def client_fn(context: Context)`.The provided `client_fn` has signature: {'cid': <Parameter "cid: str">}. You can import the `Context` like this: `from flwr.common import Context`
(ClientAppActor pid=28220) 
(ClientAppActor pid=28220)             This is a deprecated feature. It will be removed
(ClientAppActor pid=28220)             entirely in future versions of Flower.
(ClientAppActor pid=28220)         
(ClientAppActor pid=28220) c:\Users\aitoo\anaconda3\envs\fl_env\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
(ClientAppActor pid=28220)   warnings.warn(
(ClientAppActor pid=28220) c:\Users\aitoo\anaconda3\envs\fl_env\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated si

...

(ClientAppActor pid=28220) WARNING :   DEPRECATED FEATURE: `client_fn` now expects a signature `def client_fn(context: Context)`.The provided `client_fn` has signature: {'cid': <Parameter "cid: str">}. You can import the `Context` like this: `from flwr.common import Context`
(ClientAppActor pid=28220) 
(ClientAppActor pid=28220)             This is a deprecated feature. It will be removed
(ClientAppActor pid=28220)             entirely in future versions of Flower.
(ClientAppActor pid=28220)         
(ClientAppActor pid=28220) c:\Users\aitoo\anaconda3\envs\fl_env\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
(ClientAppActor pid=28220)   warnings.warn(
(ClientAppActor pid=28220) c:\Users\aitoo\anaconda3\envs\fl_env\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated si

(ClientAppActor pid=28220) Client 2 - Round 1 - Loss: 0.0071 | Acc: 0.9893 | Prec: 0.9946 | Rec: 1.0000 | F1: 0.9964
(ClientAppActor pid=28220) Client 4 - Round 1 - Loss: 0.0028 | Acc: 1.0000 | Prec: 1.0000 | Rec: 1.0000 | F1: 1.0000


(ClientAppActor pid=28220) WARNING :   DEPRECATED FEATURE: `client_fn` now expects a signature `def client_fn(context: Context)`.The provided `client_fn` has signature: {'cid': <Parameter "cid: str">}. You can import the `Context` like this: `from flwr.common import Context`
(ClientAppActor pid=28220) 
(ClientAppActor pid=28220)             This is a deprecated feature. It will be removed
(ClientAppActor pid=28220)             entirely in future versions of Flower.
(ClientAppActor pid=28220)         
(ClientAppActor pid=28220) c:\Users\aitoo\anaconda3\envs\fl_env\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
(ClientAppActor pid=28220)   warnings.warn(
(ClientAppActor pid=28220) c:\Users\aitoo\anaconda3\envs\fl_env\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated si

(ClientAppActor pid=28220) Client 0 - Round 1 - Loss: 0.0043 | Acc: 0.9929 | Prec: 0.9970 | Rec: 1.0000 | F1: 0.9981


(ClientAppActor pid=28220) WARNING :   Deprecation Warning: The `client_fn` function must return an instance of `Client`, but an instance of `NumpyClient` was returned. Please use `NumPyClient.to_client()` method to convert it to `Client`.
(ClientAppActor pid=28220) WARNING :   DEPRECATED FEATURE: `client_fn` now expects a signature `def client_fn(context: Context)`.The provided `client_fn` has signature: {'cid': <Parameter "cid: str">}. You can import the `Context` like this: `from flwr.common import Context`
(ClientAppActor pid=28220) 
(ClientAppActor pid=28220)             This is a deprecated feature. It will be removed
(ClientAppActor pid=28220)             entirely in future versions of Flower.
(ClientAppActor pid=28220)         
(ClientAppActor pid=28220) c:\Users\aitoo\anaconda3\envs\fl_env\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
(ClientAp

In [ ]:
########

In [ ]:
# CNN with attention
# ==============================
# Federated Ocular Disease Classifier with Enhanced CNN + Attention
# ==============================

import os
import pandas as pd
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision import transforms as T
from PIL import Image
import flwr as fl
from sklearn.metrics import precision_score, recall_score, f1_score, hamming_loss
import ast
import numpy as np

# Paths
base_path = "C:/Users/aitoo/OneDrive/Desktop/FYP-Code/ODIR-5K/ODIR-5K"
train_images_dir = os.path.join(base_path, "Training Images")
data_file = os.path.join(base_path, "data.xlsx")

# Load data
df = pd.read_excel(data_file)
label_cols = ["N", "D", "G", "C", "A", "H", "M", "O"]
rows = []
for _, r in df.iterrows():
    rows.append({"filename": r["Left-Fundus"], "labels": r[label_cols].values.astype(int).tolist()})
    rows.append({"filename": r["Right-Fundus"], "labels": r[label_cols].values.astype(int).tolist()})
df_images = pd.DataFrame(rows)
df_images.to_csv("C:/Users/aitoo/OneDrive/Desktop/FYP-Code/first_raw/ocular_labels.csv", index=False)

# Reload and parse labels
df_images = pd.read_csv("C:/Users/aitoo/OneDrive/Desktop/FYP-Code/first_raw/ocular_labels.csv")
df_images["labels"] = df_images["labels"].apply(ast.literal_eval)

# Dataset
class OcularDataset(Dataset):
    def __init__(self, dataframe, image_dir, transform=None):
        self.df = dataframe.reset_index(drop=True)
        self.image_dir = image_dir
        self.transform = transform if transform else T.Compose([
            T.Resize((224, 224)),
            T.ToTensor(),
            T.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ])

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        img_path = os.path.join(self.image_dir, row['filename'])
        img = Image.open(img_path).convert("RGB")
        labels = torch.tensor(row["labels"], dtype=torch.float32)
        return self.transform(img), labels

# Split dataset into 5 clients
total_len = len(df_images)
split_size = total_len // 5
splits = [split_size] * 5
splits[-1] += total_len - sum(splits)
clients_dfs = random_split(df_images, splits)

# Model: Enhanced CNN + Attention
class SEBlock(nn.Module):
    def __init__(self, channel, reduction=16):
        super(SEBlock, self).__init__()
        self.avg_pool = nn.AdaptiveAvgPool2d(1)
        self.fc = nn.Sequential(
            nn.Linear(channel, channel // reduction, bias=False),
            nn.ReLU(inplace=True),
            nn.Linear(channel // reduction, channel, bias=False),
            nn.Sigmoid()
        )

    def forward(self, x):
        b, c, _, _ = x.size()
        y = self.avg_pool(x).view(b, c)
        y = self.fc(y).view(b, c, 1, 1)
        return x * y.expand_as(x)

class CNNWithAttention(nn.Module):
    def __init__(self, num_classes=8):
        super(CNNWithAttention, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, stride=2, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            SEBlock(64),
            nn.Conv2d(64, 128, kernel_size=3, stride=2, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
            SEBlock(128),
            nn.Conv2d(128, 256, kernel_size=3, stride=2, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True),
            SEBlock(256),
            nn.AdaptiveAvgPool2d((1, 1))
        )
        self.dropout = nn.Dropout(0.3)
        self.classifier = nn.Linear(256, num_classes)

    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), -1)
        x = self.dropout(x)
        return self.classifier(x)

def get_model():
    return CNNWithAttention()

# Device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Client class
class FlowerClient(fl.client.NumPyClient):
    def __init__(self, model, train_loader, val_loader, client_id):
        self.model = model.to(device)
        self.train_loader = train_loader
        self.val_loader = val_loader
        pos_weight = torch.tensor([3.0] * 8).to(device)
        self.criterion = nn.BCEWithLogitsLoss(pos_weight=pos_weight)
        self.optimizer = torch.optim.Adam(self.model.parameters(), lr=1e-3)
        self.client_id = client_id
        self.round_counter = 0

    def get_parameters(self, config):
        return [val.cpu().numpy() for _, val in self.model.state_dict().items()]

    def set_parameters(self, parameters):
        state_dict = self.model.state_dict()
        new_state_dict = {k: torch.tensor(v).to(device) for k, v in zip(state_dict.keys(), parameters)}
        self.model.load_state_dict(new_state_dict)

    def fit(self, parameters, config):
        self.set_parameters(parameters)
        self.model.train()
        for epoch in range(10):
            for images, labels in self.train_loader:
                images, labels = images.to(device), labels.to(device)
                self.optimizer.zero_grad()
                outputs = self.model(images)
                loss = self.criterion(outputs, labels)
                loss.backward()
                self.optimizer.step()
        return self.get_parameters(config={}), len(self.train_loader.dataset), {}

    def evaluate(self, parameters, config):
        self.set_parameters(parameters)
        self.model.eval()
        y_true, y_pred = [], []
        val_loss = 0
        with torch.no_grad():
            for images, labels in self.val_loader:
                images, labels = images.to(device), labels.to(device)
                outputs = self.model(images)
                val_loss += self.criterion(outputs, labels).item()
                preds = torch.sigmoid(outputs).cpu().numpy()
                y_true.extend(labels.cpu().numpy())
                y_pred.extend((preds > 0.2).astype(int))  # Lower threshold

        y_true = np.array(y_true)
        y_pred = np.array(y_pred)

        # Compute metrics
        per_label_accuracy = (y_true == y_pred).mean()
        hamming = hamming_loss(y_true, y_pred)
        prec = precision_score(y_true, y_pred, average="samples", zero_division=0)
        rec = recall_score(y_true, y_pred, average="samples", zero_division=0)
        f1 = f1_score(y_true, y_pred, average="samples", zero_division=0)

        self.round_counter += 1

        metrics_row = {
            "client": self.client_id,
            "round": self.round_counter,
            "loss": val_loss / len(self.val_loader),
            "per_label_accuracy": per_label_accuracy,
            "hamming_loss": hamming,
            "precision": prec,
            "recall": rec,
            "f1": f1
        }

        print(
            f"Client {self.client_id} - Round {self.round_counter} - Loss: {metrics_row['loss']:.4f} "
            f"| Per-Label Acc: {per_label_accuracy:.4f} | Hamming: {hamming:.4f} "
            f"| Prec: {prec:.4f} | Rec: {rec:.4f} | F1: {f1:.4f}"
        )

        client_csv_path = f"C:/Users/aitoo/OneDrive/Desktop/FYP-Code/first_raw/FL_Client_{self.client_id}.csv"
        pd.DataFrame([metrics_row]).to_csv(
            client_csv_path,
            mode="a",
            header=not os.path.exists(client_csv_path),
            index=False,
        )

        return float(metrics_row["loss"]), len(self.val_loader.dataset), {}

# Client factory
def client_fn(cid: str):
    idx = int(cid)
    client_df = clients_dfs[idx].dataset.iloc[clients_dfs[idx].indices]
    dataset = OcularDataset(client_df, train_images_dir)
    split = int(0.8 * len(dataset))
    train_ds, val_ds = torch.utils.data.random_split(dataset, [split, len(dataset) - split])
    train_loader = DataLoader(train_ds, batch_size=8, shuffle=True)
    val_loader = DataLoader(val_ds, batch_size=8)
    return FlowerClient(get_model(), train_loader, val_loader, client_id=cid)

# Run simulation
strategy = fl.server.strategy.FedAvg(
    fraction_fit=1.0,
    min_fit_clients=5,
    min_available_clients=5,
)

fl.simulation.start_simulation(
    client_fn=client_fn,
    num_clients=5,
    client_resources={
        "num_cpus": 1,
        "num_gpus": 1.0
    },
    strategy=strategy,
    config=fl.server.ServerConfig(num_rounds=50)
)


c:\Users\aitoo\anaconda3\envs\fl_env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-07-04 22:17:29,872	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
	Instead, use the `flwr run` CLI command to start a local simulation in your Flower app, as shown for example below:

		$ flwr new  # Create a new Flower app from a template

		$ flwr run  # Run the Flower app in Simulation Mode

	Using `start_simulation()` is deprecated.

            This is a deprecated feature. It will be removed
            entirely in future versions of Flower.
        
INFO :      Starting Flower simulation, config: num_rounds=50, no round_timeout
2025-07-04 22:17:34,512	INFO worker.py:1771 -- Started a local Ray instance.
INFO :      Flower

(ClientAppActor pid=5296) Client 2 - Round 1 - Loss: 0.7432 | Per-Label Acc: 0.4589 | Hamming: 0.5411 | Prec: 0.2057 | Rec: 0.9006 | F1: 0.3290


(ClientAppActor pid=5296) WARNING :   DEPRECATED FEATURE: `client_fn` now expects a signature `def client_fn(context: Context)`.The provided `client_fn` has signature: {'cid': <Parameter "cid: str">}. You can import the `Context` like this: `from flwr.common import Context`
(ClientAppActor pid=5296) 
(ClientAppActor pid=5296)             This is a deprecated feature. It will be removed
(ClientAppActor pid=5296)             entirely in future versions of Flower.
(ClientAppActor pid=5296)         
(ClientAppActor pid=5296) WARNING :   Deprecation Warning: The `client_fn` function must return an instance of `Client`, but an instance of `NumpyClient` was returned. Please use `NumPyClient.to_client()` method to convert it to `Client`.
(ClientAppActor pid=5296) WARNING :   DEPRECATED FEATURE: `client_fn` now expects a signature `def client_fn(context: Context)`.The provided `client_fn` has signature: {'cid': <Parameter "cid: str">}. You can import the `Context` like this: `from flwr.common i

(ClientAppActor pid=5296) Client 0 - Round 1 - Loss: 0.7170 | Per-Label Acc: 0.4629 | Hamming: 0.5371 | Prec: 0.2092 | Rec: 0.9250 | F1: 0.3353


(ClientAppActor pid=5296) WARNING :   DEPRECATED FEATURE: `client_fn` now expects a signature `def client_fn(context: Context)`.The provided `client_fn` has signature: {'cid': <Parameter "cid: str">}. You can import the `Context` like this: `from flwr.common import Context`
(ClientAppActor pid=5296) 
(ClientAppActor pid=5296)             This is a deprecated feature. It will be removed
(ClientAppActor pid=5296)             entirely in future versions of Flower.
(ClientAppActor pid=5296)         
(ClientAppActor pid=5296) WARNING :   Deprecation Warning: The `client_fn` function must return an instance of `Client`, but an instance of `NumpyClient` was returned. Please use `NumPyClient.to_client()` method to convert it to `Client`.


...

(ClientAppActor pid=5296) WARNING :   DEPRECATED FEATURE: `client_fn` now expects a signature `def client_fn(context: Context)`.The provided `client_fn` has signature: {'cid': <Parameter "cid: str">}. You can import the `Context` like this: `from flwr.common import Context`
(ClientAppActor pid=5296) 
(ClientAppActor pid=5296)             This is a deprecated feature. It will be removed
(ClientAppActor pid=5296)             entirely in future versions of Flower.
(ClientAppActor pid=5296)         
(ClientAppActor pid=5296) WARNING :   Deprecation Warning: The `client_fn` function must return an instance of `Client`, but an instance of `NumpyClient` was returned. Please use `NumPyClient.to_client()` method to convert it to `Client`.


(ClientAppActor pid=5296) Client 3 - Round 1 - Loss: 0.4951 | Per-Label Acc: 0.6978 | Hamming: 0.3022 | Prec: 0.3496 | Rec: 0.9464 | F1: 0.4879


INFO :      aggregate_evaluate: received 5 results and 0 failures
INFO :      
INFO :      [ROUND 41]
INFO :      configure_fit: strategy sampled 5 clients (out of 5)
(ClientAppActor pid=5296) WARNING :   DEPRECATED FEATURE: `client_fn` now expects a signature `def client_fn(context: Context)`.The provided `client_fn` has signature: {'cid': <Parameter "cid: str">}. You can import the `Context` like this: `from flwr.common import Context`
(ClientAppActor pid=5296) 
(ClientAppActor pid=5296)             This is a deprecated feature. It will be removed
(ClientAppActor pid=5296)             entirely in future versions of Flower.
(ClientAppActor pid=5296)         
(ClientAppActor pid=5296) WARNING :   Deprecation Warning: The `client_fn` function must return an instance of `Client`, but an instance of `NumpyClient` was returned. Please use `NumPyClient.to_client()` method to convert it to `Client`.


(ClientAppActor pid=5296) Client 1 - Round 1 - Loss: 0.5384 | Per-Label Acc: 0.7009 | Hamming: 0.2991 | Prec: 0.3440 | Rec: 0.9351 | F1: 0.4882


(ClientAppActor pid=5296) WARNING :   DEPRECATED FEATURE: `client_fn` now expects a signature `def client_fn(context: Context)`.The provided `client_fn` has signature: {'cid': <Parameter "cid: str">}. You can import the `Context` like this: `from flwr.common import Context`
(ClientAppActor pid=5296) 
(ClientAppActor pid=5296)             This is a deprecated feature. It will be removed
(ClientAppActor pid=5296)             entirely in future versions of Flower.
(ClientAppActor pid=5296)         
(ClientAppActor pid=5296) WARNING :   Deprecation Warning: The `client_fn` function must return an instance of `Client`, but an instance of `NumpyClient` was returned. Please use `NumPyClient.to_client()` method to convert it to `Client`.
(ClientAppActor pid=5296) WARNING :   DEPRECATED FEATURE: `client_fn` now expects a signature `def client_fn(context: Context)`.The provided `client_fn` has signature: {'cid': <Parameter "cid: str">}. You can import the `Context` like this: `from flwr.common i

In [ ]:
############

In [ ]:
# Swin Transformer
# ==============================
# Federated Ocular Disease Classifier with Swin Transformer
# - Weighted loss
# - Lower threshold
# - Per-client and global CSV logging
# ==============================

import os
import pandas as pd
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision import transforms as T
from PIL import Image
import flwr as fl
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import ast
import timm

# Paths
base_path = "C:/Users/aitoo/OneDrive/Desktop/FYP-Code/ODIR-5K/ODIR-5K"
train_images_dir = os.path.join(base_path, "Training Images")
data_file = os.path.join(base_path, "data.xlsx")

# Load data
df = pd.read_excel(data_file)
label_cols = ["N", "D", "G", "C", "A", "H", "M", "O"]
rows = []
for _, r in df.iterrows():
    rows.append({"filename": r["Left-Fundus"], "labels": r[label_cols].values.astype(int).tolist()})
    rows.append({"filename": r["Right-Fundus"], "labels": r[label_cols].values.astype(int).tolist()})
df_images = pd.DataFrame(rows)
df_images.to_csv("C:/Users/aitoo/OneDrive/Desktop/FYP-Code/first_raw/ocular_labels.csv", index=False)

# Reload and parse labels
df_images = pd.read_csv("C:/Users/aitoo/OneDrive/Desktop/FYP-Code/first_raw/ocular_labels.csv")
df_images["labels"] = df_images["labels"].apply(ast.literal_eval)

# Dataset
class OcularDataset(Dataset):
    def __init__(self, dataframe, image_dir, transform=None):
        self.df = dataframe.reset_index(drop=True)
        self.image_dir = image_dir
        self.transform = transform if transform else T.Compose([
            T.Resize((224, 224)),
            T.ToTensor(),
            T.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ])

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        img_path = os.path.join(self.image_dir, row['filename'])
        img = Image.open(img_path).convert("RGB")
        labels = torch.tensor(row["labels"], dtype=torch.float32)
        return self.transform(img), labels

# Split dataset into 5 clients
total_len = len(df_images)
split_size = total_len // 5
splits = [split_size] * 5
splits[-1] += total_len - sum(splits)
clients_dfs = random_split(df_images, splits)

# Model
def get_model():
    model = timm.create_model("swin_base_patch4_window7_224", pretrained=True, num_classes=8)
    return model

# Device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Client class
class FlowerClient(fl.client.NumPyClient):
    def __init__(self, model, train_loader, val_loader, client_id):
        self.model = model.to(device)
        self.train_loader = train_loader
        self.val_loader = val_loader
        pos_weight = torch.tensor([3.0] * 8).to(device)
        self.criterion = nn.BCEWithLogitsLoss(pos_weight=pos_weight)
        self.optimizer = torch.optim.Adam(self.model.parameters(), lr=1e-4)
        self.client_id = client_id
        self.round_counter = 0

    def get_parameters(self, config):
        return [val.cpu().numpy() for _, val in self.model.state_dict().items()]

    def set_parameters(self, parameters):
        state_dict = self.model.state_dict()
        new_state_dict = {k: torch.tensor(v).to(device) for k, v in zip(state_dict.keys(), parameters)}
        self.model.load_state_dict(new_state_dict)

    def fit(self, parameters, config):
        self.set_parameters(parameters)
        self.model.train()
        for epoch in range(3):
            for images, labels in self.train_loader:
                images, labels = images.to(device), labels.to(device)
                self.optimizer.zero_grad()
                outputs = self.model(images)
                if isinstance(outputs, tuple):
                    outputs = outputs[0]
                loss = self.criterion(outputs, labels)
                loss.backward()
                self.optimizer.step()
        return self.get_parameters(config={}), len(self.train_loader.dataset), {}

    def evaluate(self, parameters, config):
        self.set_parameters(parameters)
        self.model.eval()
        y_true, y_pred = [], []
        val_loss = 0
        with torch.no_grad():
            for images, labels in self.val_loader:
                images, labels = images.to(device), labels.to(device)
                outputs = self.model(images)
                if isinstance(outputs, tuple):
                    outputs = outputs[0]
                val_loss += self.criterion(outputs, labels).item()
                preds = torch.sigmoid(outputs).cpu().numpy()
                y_true.extend(labels.cpu().numpy())
                y_pred.extend((preds > 0.3).astype(int))

        acc = accuracy_score(y_true, y_pred)
        prec = precision_score(y_true, y_pred, average="samples", zero_division=0)
        rec = recall_score(y_true, y_pred, average="samples", zero_division=0)
        f1 = f1_score(y_true, y_pred, average="samples", zero_division=0)

        self.round_counter += 1

        metrics_row = {
            "client": self.client_id,
            "round": self.round_counter,
            "loss": val_loss / len(self.val_loader),
            "accuracy": acc,
            "precision": prec,
            "recall": rec,
            "f1": f1
        }

        print(
            f"Client {self.client_id} - Round {self.round_counter} - Loss: {metrics_row['loss']:.4f} | Acc: {acc:.4f} | Prec: {prec:.4f} | Rec: {rec:.4f} | F1: {f1:.4f}",
            flush=True
        )

        client_csv_path = f"C:/Users/aitoo/OneDrive/Desktop/FYP-Code/first_raw/FL_Client_{self.client_id}.csv"
        pd.DataFrame([metrics_row]).to_csv(
            client_csv_path,
            mode="a",
            header=not os.path.exists(client_csv_path),
            index=False,
        )

        return float(metrics_row["loss"]), len(self.val_loader.dataset), {}

# Client factory
def client_fn(cid: str):
    idx = int(cid)
    client_df = clients_dfs[idx].dataset.iloc[clients_dfs[idx].indices]
    dataset = OcularDataset(client_df, train_images_dir)
    split = int(0.8 * len(dataset))
    train_ds, val_ds = torch.utils.data.random_split(dataset, [split, len(dataset) - split])
    train_loader = DataLoader(train_ds, batch_size=4, shuffle=True)
    val_loader = DataLoader(val_ds, batch_size=4)
    return FlowerClient(get_model(), train_loader, val_loader, client_id=cid)

# Custom strategy with global evaluation
class MyFedAvg(fl.server.strategy.FedAvg):
    def evaluate(self, server_round, parameters):
        model = get_model().to(device)
        state_dict = model.state_dict()
        new_state_dict = {k: torch.tensor(v).to(device) for k, v in zip(state_dict.keys(), parameters)}
        model.load_state_dict(new_state_dict)
        model.eval()
        all_y_true, all_y_pred = [], []
        total_loss = 0
        with torch.no_grad():
            for idx in range(5):
                client_df = clients_dfs[idx].dataset.iloc[clients_dfs[idx].indices]
                dataset = OcularDataset(client_df, train_images_dir)
                _, val_ds = torch.utils.data.random_split(dataset, [int(0.8 * len(dataset)), len(dataset) - int(0.8 * len(dataset))])
                val_loader = DataLoader(val_ds, batch_size=4)
                for images, labels in val_loader:
                    images, labels = images.to(device), labels.to(device)
                    outputs = model(images)
                    if isinstance(outputs, tuple):
                        outputs = outputs[0]
                    total_loss += nn.BCEWithLogitsLoss()(outputs, labels).item()
                    preds = torch.sigmoid(outputs).cpu().numpy()
                    all_y_true.extend(labels.cpu().numpy())
                    all_y_pred.extend((preds > 0.3).astype(int))

        acc = accuracy_score(all_y_true, all_y_pred)
        prec = precision_score(all_y_true, all_y_pred, average="samples", zero_division=0)
        rec = recall_score(all_y_true, all_y_pred, average="samples", zero_division=0)
        f1 = f1_score(all_y_true, all_y_pred, average="samples", zero_division=0)

        metrics_row = {
            "round": server_round,
            "loss": total_loss / len(all_y_true),
            "accuracy": acc,
            "precision": prec,
            "recall": rec,
            "f1": f1
        }

        print(
            f"GLOBAL - Round {server_round} - Loss: {metrics_row['loss']:.4f} | Acc: {acc:.4f} | Prec: {prec:.4f} | Rec: {rec:.4f} | F1: {f1:.4f}",
            flush=True
        )

        global_csv_path = "C:/Users/aitoo/OneDrive/Desktop/FYP-Code/first_raw/FL_Global.csv"
        pd.DataFrame([metrics_row]).to_csv(
            global_csv_path,
            mode="a",
            header=not os.path.exists(global_csv_path),
            index=False,
        )

        return float(metrics_row["loss"]), {}

# Run simulation


strategy = fl.server.strategy.FedAvg(
    fraction_fit=1.0,
    min_fit_clients=5,
    min_available_clients=5,
)

fl.simulation.start_simulation(
    client_fn=client_fn,
    num_clients=5,
    client_resources={
        "num_cpus": 1,
        "num_gpus": 1.0
    },
    strategy=strategy,
    config=fl.server.ServerConfig(num_rounds=50)
)



c:\Users\aitoo\anaconda3\envs\fl_env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-07-09 10:44:53,096	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
c:\Users\aitoo\anaconda3\envs\fl_env\Lib\site-packages\wandb\util.py:118: SentryHubDeprecationWarning: `sentry_sdk.Hub` is deprecated and will be removed in a future major release. Please consult our 1.x to 2.x migration guide for details on how to migrate `Hub` usage to the new API: https://docs.sentry.io/platforms/python/migration/1.x-to-2.x
  sentry_hub = sentry_sdk.Hub(sentry_client)
c:\Users\aitoo\anaconda3\envs\fl_env\Lib\site-packages\wandb\sdk\internal\internal_api.py:30: DeprecationWarning: pkg_resources is deprecated as an API. See https://setuptools.pyp

(ClientAppActor pid=12452) Client 1 - Round 1 - Loss: 0.5969 | Acc: 0.0000 | Prec: 0.3280 | Rec: 0.8762 | F1: 0.4689


(ClientAppActor pid=12452) WARNING :   DEPRECATED FEATURE: `client_fn` now expects a signature `def client_fn(context: Context)`.The provided `client_fn` has signature: {'cid': <Parameter "cid: str">}. You can import the `Context` like this: `from flwr.common import Context`
(ClientAppActor pid=12452) 
(ClientAppActor pid=12452)             This is a deprecated feature. It will be removed
(ClientAppActor pid=12452)             entirely in future versions of Flower.
(ClientAppActor pid=12452)         
(ClientAppActor pid=12452) WARNING :   Deprecation Warning: The `client_fn` function must return an instance of `Client`, but an instance of `NumpyClient` was returned. Please use `NumPyClient.to_client()` method to convert it to `Client`.


(ClientAppActor pid=12452) Client 0 - Round 1 - Loss: 0.6197 | Acc: 0.0000 | Prec: 0.3274 | Rec: 0.8542 | F1: 0.4632


(ClientAppActor pid=12452) WARNING :   DEPRECATED FEATURE: `client_fn` now expects a signature `def client_fn(context: Context)`.The provided `client_fn` has signature: {'cid': <Parameter "cid: str">}. You can import the `Context` like this: `from flwr.common import Context`
(ClientAppActor pid=12452) 
(ClientAppActor pid=12452)             This is a deprecated feature. It will be removed
(ClientAppActor pid=12452)             entirely in future versions of Flower.
(ClientAppActor pid=12452)         
(ClientAppActor pid=12452) WARNING :   Deprecation Warning: The `client_fn` function must return an instance of `Client`, but an instance of `NumpyClient` was returned. Please use `NumPyClient.to_client()` method to convert it to `Client`.


...

(ClientAppActor pid=12452) WARNING :   DEPRECATED FEATURE: `client_fn` now expects a signature `def client_fn(context: Context)`.The provided `client_fn` has signature: {'cid': <Parameter "cid: str">}. You can import the `Context` like this: `from flwr.common import Context`
(ClientAppActor pid=12452) 
(ClientAppActor pid=12452)             This is a deprecated feature. It will be removed
(ClientAppActor pid=12452)             entirely in future versions of Flower.
(ClientAppActor pid=12452)         
(ClientAppActor pid=12452) WARNING :   Deprecation Warning: The `client_fn` function must return an instance of `Client`, but an instance of `NumpyClient` was returned. Please use `NumPyClient.to_client()` method to convert it to `Client`.


(ClientAppActor pid=12452) Client 4 - Round 1 - Loss: 0.5793 | Acc: 0.0036 | Prec: 0.3376 | Rec: 0.8982 | F1: 0.4829


(ClientAppActor pid=12452) WARNING :   DEPRECATED FEATURE: `client_fn` now expects a signature `def client_fn(context: Context)`.The provided `client_fn` has signature: {'cid': <Parameter "cid: str">}. You can import the `Context` like this: `from flwr.common import Context`
(ClientAppActor pid=12452) 
(ClientAppActor pid=12452)             This is a deprecated feature. It will be removed
(ClientAppActor pid=12452)             entirely in future versions of Flower.
(ClientAppActor pid=12452)         
(ClientAppActor pid=12452) WARNING :   Deprecation Warning: The `client_fn` function must return an instance of `Client`, but an instance of `NumpyClient` was returned. Please use `NumPyClient.to_client()` method to convert it to `Client`.
INFO :      aggregate_evaluate: received 5 results and 0 failures
INFO :      
INFO :      [ROUND 2]
INFO :      configure_fit: strategy sampled 5 clients (out of 5)


(ClientAppActor pid=12452) Client 2 - Round 1 - Loss: 0.5901 | Acc: 0.0036 | Prec: 0.3340 | Rec: 0.8780 | F1: 0.4739


(ClientAppActor pid=12452) WARNING :   DEPRECATED FEATURE: `client_fn` now expects a signature `def client_fn(context: Context)`.The provided `client_fn` has signature: {'cid': <Parameter "cid: str">}. You can import the `Context` like this: `from flwr.common import Context`
(ClientAppActor pid=12452) 
(ClientAppActor pid=12452)             This is a deprecated feature. It will be removed
(ClientAppActor pid=12452)             entirely in future versions of Flower.
(ClientAppActor pid=12452)         
(ClientAppActor pid=12452) WARNING :   Deprecation Warning: The `client_fn` function must return an instance of `Client`, but an instance of `NumpyClient` was returned. Please use `NumPyClient.to_client()` method to convert it to `Client`.


In [ ]:
################